In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline 

In [ ]:
batch_size = 50
seq_len = 20
learning_rate = 1e-4
epoch = 2

In [2]:
# time dependent list of discrete variables of shape (B,T)
disc_T = ['dayOfWeek','payDay','month','earthquake','type','locale','locale_name','transferred','onpromotion']
# time independent list of discrete variables of shape of shape (B,)
disc_X = ['store_nbr','item_nbr',
 'family', 'class',
 'perishable', 'city',
 'state', 'type', 'cluster']

In [4]:
cardinalitys_X = [55, 3998, 34, 256, 2, 23, 17, 6, 18]
cardinalitys_T = [7, 2, 13, 2, 7, 4, 25, 2, 3]

In [7]:
dimentions_X = [2, 50, 1, 2, 1, 1, 1, 1, 1]
dimentions_T = [1, 1, 1, 1, 1, 1, 1, 1, 1]

In [ ]:
trainRNN = pd.read_csv('trainRNN.csv',engine='python')
valRNN = pd.read_csv('valRNN.csv',engine='python')

In [ ]:
with open(r"dateVar.pickle", "rb") as input_file:
    dateVar_list = cPickle.load(input_file)